In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Python script to calculate MMC properties using Rule of Mixtures

def calculate_mmc_properties(matrix, reinforcements):
    """
    Calculate the effective properties of an MMC using Rule of Mixtures.
    :param matrix: Dictionary containing matrix properties {density, modulus, conductivity}.
    :param reinforcements: List of dictionaries for reinforcement properties [{density, modulus, conductivity, vol_fraction}, ...].
    :return: Dictionary with calculated MMC properties.
    """
    # Initialize properties with matrix contribution
    density_mmc = matrix['density'] * (1 - sum(r['vol_fraction'] for r in reinforcements))
    modulus_mmc = matrix['modulus'] * (1 - sum(r['vol_fraction'] for r in reinforcements))
    conductivity_mmc = matrix['conductivity'] * (1 - sum(r['vol_fraction'] for r in reinforcements))
    
    # Add contribution from reinforcements
    for r in reinforcements:
        density_mmc += r['density'] * r['vol_fraction']
        modulus_mmc += r['modulus'] * r['vol_fraction']
        conductivity_mmc += r['conductivity'] * r['vol_fraction']
    
    return {
        'Density (kg/m³)': round(density_mmc, 2),
        'Young\'s Modulus (GPa)': round(modulus_mmc, 2),
        'Thermal Conductivity (W/mK)': round(conductivity_mmc, 2)
    }

# Define Material Properties
matrix_material = {
    'density': 2700,  # kg/m³ (Al6061)
    'modulus': 70,    # GPa
    'conductivity': 167  # W/mK
}

reinforcements = [
    {'density': 2520, 'modulus': 450, 'conductivity': 29, 'vol_fraction': 0.05},  # B4C (5%)
    {'density': 1350, 'modulus': 12, 'conductivity': 0.5, 'vol_fraction': 0.05}   # Biochar (5%)
]

# Compute MMC Properties
mmc_properties = calculate_mmc_properties(matrix_material, reinforcements)

# Display Results
for prop, value in mmc_properties.items():
    print(f"{prop}: {value}")


Density (kg/m³): 2623.5
Young's Modulus (GPa): 86.1
Thermal Conductivity (W/mK): 151.78


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Sample dataset (Replace with real experimental data)
data = {
    'Al6061_fraction': [1.0, 0.95, 0.95, 0.95, 0.90],
    'B4C_fraction': [0.0, 0.05, 0.0, 0.04, 0.04],
    'Biochar_fraction': [0.0, 0.0, 0.05, 0.01, 0.06],
    'Density': [2700, 2685, 2557.5, 2632.5, 2562],
    'Youngs_Modulus': [70, 88, 66.4, 87.6, 79.2],
    'Thermal_Conductivity': [167, 160.4, 151, 157.5, 149],
    'Tensile_Strength': [310, 350, 290, 340, 330]  # Target variable
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Define features (X) and target variable (y)
X = df[['Al6061_fraction', 'B4C_fraction', 'Biochar_fraction', 'Density', 'Youngs_Modulus', 'Thermal_Conductivity']]
y = df['Tensile_Strength']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Model Performance:")
print(f"Mean Squared Error: {mse:.2f}")
print(f"R² Score: {r2:.2f}")

# Function to predict tensile strength for new compositions
def predict_strength(al6061, b4c, biochar, density, modulus, conductivity):
    input_data = np.array([[al6061, b4c, biochar, density, modulus, conductivity]])
    return model.predict(input_data)[0]

# Example Prediction
new_sample = [0.92, 0.05, 0.03, 2600, 85, 155]  # Modify as needed
predicted_strength = predict_strength(*new_sample)
print(f"Predicted Tensile Strength: {predicted_strength:.2f} MPa")


Model Performance:
Mean Squared Error: 392.04
R² Score: nan
Predicted Tensile Strength: 332.00 MPa


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Sample dataset (Replace with real experimental data)
data = {
    'Al6061_fraction': [1.0, 0.95, 0.95, 0.95, 0.90],
    'B4C_fraction': [0.0, 0.05, 0.00, 0.04, 0.04],
    'Biochar_fraction': [0.0, 0.0, 0.05, 0.01, 0.06],
    'Density': [2700, 2685, 2557.5, 2632.5, 2562],
    'Youngs_Modulus': [70, 88, 66.4, 87.6, 79.2],
    'Thermal_Conductivity': [167, 160.4, 151, 157.5, 149],
    'Tensile_Strength': [310, 350, 290, 340, 330],
    'Compression_Strength': [420, 460, 400, 450, 440],
    'Impact_Strength': [35, 40, 30, 38, 37],
    'Flexural_Strength': [500, 540, 480, 530, 520]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Define features (X) and target variables (y)
X = df[['Al6061_fraction', 'B4C_fraction', 'Biochar_fraction']]
y_density = df['Density']
y_modulus = df['Youngs_Modulus']
y_conductivity = df['Thermal_Conductivity']
y_tensile = df['Tensile_Strength']
y_compression = df['Compression_Strength']
y_impact = df['Impact_Strength']
y_flexural = df['Flexural_Strength']

# Split data into training and testing sets
X_train, X_test, y_density_train, y_density_test = train_test_split(X, y_density, test_size=0.2, random_state=42)
X_train, X_test, y_modulus_train, y_modulus_test = train_test_split(X, y_modulus, test_size=0.2, random_state=42)
X_train, X_test, y_conductivity_train, y_conductivity_test = train_test_split(X, y_conductivity, test_size=0.2, random_state=42)
X_train, X_test, y_tensile_train, y_tensile_test = train_test_split(X, y_tensile, test_size=0.2, random_state=42)
X_train, X_test, y_compression_train, y_compression_test = train_test_split(X, y_compression, test_size=0.2, random_state=42)
X_train, X_test, y_impact_train, y_impact_test = train_test_split(X, y_impact, test_size=0.2, random_state=42)
X_train, X_test, y_flexural_train, y_flexural_test = train_test_split(X, y_flexural, test_size=0.2, random_state=42)

# Train Random Forest models
model_density = RandomForestRegressor(n_estimators=100, random_state=42)
model_modulus = RandomForestRegressor(n_estimators=100, random_state=42)
model_conductivity = RandomForestRegressor(n_estimators=100, random_state=42)
model_tensile = RandomForestRegressor(n_estimators=100, random_state=42)
model_compression = RandomForestRegressor(n_estimators=100, random_state=42)
model_impact = RandomForestRegressor(n_estimators=100, random_state=42)
model_flexural = RandomForestRegressor(n_estimators=100, random_state=42)

model_density.fit(X_train, y_density_train)
model_modulus.fit(X_train, y_modulus_train)
model_conductivity.fit(X_train, y_conductivity_train)
model_tensile.fit(X_train, y_tensile_train)
model_compression.fit(X_train, y_compression_train)
model_impact.fit(X_train, y_impact_train)
model_flexural.fit(X_train, y_flexural_train)

# Function to predict mechanical properties
def predict_properties(al6061, b4c, biochar):
    input_data = np.array([[al6061, b4c, biochar]])
    density = model_density.predict(input_data)[0]
    modulus = model_modulus.predict(input_data)[0]
    conductivity = model_conductivity.predict(input_data)[0]
    tensile = model_tensile.predict(input_data)[0]
    compression = model_compression.predict(input_data)[0]
    impact = model_impact.predict(input_data)[0]
    flexural = model_flexural.predict(input_data)[0]
    return density, modulus, conductivity, tensile, compression, impact, flexural

# Example Prediction
new_sample = [0.92, 0.05, 0.03]  # Modify as needed
predicted_density, predicted_modulus, predicted_conductivity, predicted_tensile, predicted_compression, predicted_impact, predicted_flexural = predict_properties(*new_sample)

print(f"Predicted Density: {predicted_density:.2f} kg/m³")
print(f"Predicted Young's Modulus: {predicted_modulus:.2f} GPa")
print(f"Predicted Thermal Conductivity: {predicted_conductivity:.2f} W/mK")
print(f"Predicted Tensile Strength: {predicted_tensile:.2f} MPa")
print(f"Predicted Compression Strength: {predicted_compression:.2f} MPa")
print(f"Predicted Impact Strength: {predicted_impact:.2f} J")
print(f"Predicted Flexural Strength: {predicted_flexural:.2f} MPa")

Predicted Density: 2608.02 kg/m³
Predicted Young's Modulus: 80.93 GPa
Predicted Thermal Conductivity: 154.75 W/mK
Predicted Tensile Strength: 330.50 MPa
Predicted Compression Strength: 440.50 MPa
Predicted Impact Strength: 36.78 J
Predicted Flexural Strength: 520.50 MPa


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Sample dataset (Replace with actual experimental data)
data = {
    'Al6061_fraction': [1.0, 0.95, 0.95, 0.95, 0.90],
    'B4C_fraction': [0.0, 0.05, 0.00, 0.04, 0.04],
    'Biochar_fraction': [0.0, 0.0, 0.05, 0.01, 0.06],
    'Density': [2700, 2685, 2557.5, 2632.5, 2562],
    'Youngs_Modulus': [70, 88, 66.4, 87.6, 79.2],
    'Thermal_Conductivity': [167, 160.4, 151, 157.5, 149],
    'Tensile_Strength': [310, 350, 290, 340, 330],
    'Compression_Strength': [420, 460, 400, 450, 440],
    'Impact_Strength': [35, 40, 30, 38, 37],
    'Flexural_Strength': [500, 540, 480, 530, 520]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Define features (X) and target variables (y)
X = df[['Al6061_fraction', 'B4C_fraction', 'Biochar_fraction']]
y_columns = ['Density', 'Youngs_Modulus', 'Thermal_Conductivity', 
             'Tensile_Strength', 'Compression_Strength', 'Impact_Strength', 'Flexural_Strength']

# Split data and train Random Forest models for each target variable
models = {}

for y_col in y_columns:
    X_train, X_test, y_train, y_test = train_test_split(X, df[y_col], test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    models[y_col] = model

# Function to predict mechanical properties for given material compositions
def predict_properties(al6061, b4c, biochar):
    input_data = np.array([[al6061, b4c, biochar]])
    predictions = {prop: model.predict(input_data)[0] for prop, model in models.items()}
    return predictions

# Predict for all 5 combinations
samples = [
    [1.0, 0.0, 0.0],   # 100% Al6061
    [0.95, 0.05, 0.0], # 95% Al6061 + 5% B4C
    [0.95, 0.0, 0.05], # 95% Al6061 + 5% Biochar
    [0.95, 0.04, 0.01],# 95% Al6061 + 4% B4C + 1% Biochar
    [0.90, 0.04, 0.06] # 90% Al6061 + 4% B4C + 6% Biochar
]

# Print predictions in tabular form
print("Predicted Mechanical and Thermal Properties:")
print("-" * 100)
print(f"{'Composition':<35}{'Density':<10}{'Youngs Modulus':<15}{'Thermal Cond.':<15}"
      f"{'Tensile':<10}{'Compression':<12}{'Impact':<10}{'Flexural':<10}")
print("-" * 100)

for i, sample in enumerate(samples):
    predictions = predict_properties(*sample)
    print(f"{i+1}. {sample}   "
          f"{predictions['Density']:.2f}     {predictions['Youngs_Modulus']:.2f}       "
          f"{predictions['Thermal_Conductivity']:.2f}      {predictions['Tensile_Strength']:.2f}      "
          f"{predictions['Compression_Strength']:.2f}     {predictions['Impact_Strength']:.2f}     "
          f"{predictions['Flexural_Strength']:.2f}")

print("-" * 100)


Predicted Mechanical and Thermal Properties:
----------------------------------------------------------------------------------------------------
Composition                        Density   Youngs Modulus Thermal Cond.  Tensile   Compression Impact    Flexural  
----------------------------------------------------------------------------------------------------
1. [1.0, 0.0, 0.0]   2670.15     69.91       163.25      306.10      416.10     33.80     496.10
2. [0.95, 0.05, 0.0]   2628.41     81.07       157.94      328.60      438.60     36.41     518.60
3. [0.95, 0.0, 0.05]   2575.97     72.26       152.70      305.00      415.00     32.61     495.00
4. [0.95, 0.04, 0.01]   2618.28     83.00       156.24      331.90      441.90     36.86     521.90
5. [0.9, 0.04, 0.06]   2571.86     80.53       150.72      330.10      440.10     35.79     520.10
----------------------------------------------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Sample dataset (Replace with real experimental data)
data = {
    'Al6061_fraction': [1.0, 0.95, 0.95, 0.95, 0.90],
    'B4C_fraction': [0.0, 0.05, 0.0, 0.04, 0.04],
    'Biochar_fraction': [0.0, 0.0, 0.05, 0.01, 0.06],
    'Density': [2670.15, 2628.41, 2575.97, 2618.28, 2571.86],
    'Youngs_Modulus': [69.91, 81.07, 72.26, 83.00, 80.53],
    'Thermal_Conductivity': [163.25, 157.94, 152.70, 156.24, 150.72],
    'Tensile_Strength': [306.10, 328.60, 305.00, 331.90, 330.10],
    'Compression_Strength': [416.10, 438.60, 415.00, 441.90, 440.10],
    'Impact_Strength': [33.80, 36.41, 32.61, 36.86, 35.79],
    'Flexural_Strength': [496.10, 518.60, 495.00, 521.90, 520.10],
    'Hardness': [75, 82, 78, 85, 80],  # Hypothetical values
    'Wear_Resistance': [0.15, 0.18, 0.14, 0.19, 0.17],  # Hypothetical values
    'Fatigue_Strength': [250, 270, 260, 280, 275]  # Hypothetical values
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Define features (X) and target variables (y)
X = df[['Density', 'Youngs_Modulus', 'Thermal_Conductivity', 'Tensile_Strength',
        'Compression_Strength', 'Impact_Strength', 'Flexural_Strength']]
y_hardness = df['Hardness']
y_wear = df['Wear_Resistance']
y_fatigue = df['Fatigue_Strength']

# Split data into training and testing sets
X_train, X_test, y_hardness_train, y_hardness_test = train_test_split(X, y_hardness, test_size=0.2, random_state=42)
X_train, X_test, y_wear_train, y_wear_test = train_test_split(X, y_wear, test_size=0.2, random_state=42)
X_train, X_test, y_fatigue_train, y_fatigue_test = train_test_split(X, y_fatigue, test_size=0.2, random_state=42)

# Train Random Forest models
model_hardness = RandomForestRegressor(n_estimators=100, random_state=42)
model_wear = RandomForestRegressor(n_estimators=100, random_state=42)
model_fatigue = RandomForestRegressor(n_estimators=100, random_state=42)

model_hardness.fit(X_train, y_hardness_train)
model_wear.fit(X_train, y_wear_train)
model_fatigue.fit(X_train, y_fatigue_train)

# Function to predict additional mechanical properties
def predict_additional_properties(density, modulus, conductivity, tensile, compression, impact, flexural):
    input_data = np.array([[density, modulus, conductivity, tensile, compression, impact, flexural]])
    hardness = model_hardness.predict(input_data)[0]
    wear_resistance = model_wear.predict(input_data)[0]
    fatigue_strength = model_fatigue.predict(input_data)[0]
    return hardness, wear_resistance, fatigue_strength

# Example Prediction
new_sample = [2650, 80, 160, 320, 430, 35, 510]  # Modify as needed
predicted_hardness, predicted_wear, predicted_fatigue = predict_additional_properties(*new_sample)

print(f"Predicted Hardness: {predicted_hardness:.2f} HV")
print(f"Predicted Wear Resistance: {predicted_wear:.3f} mm³/Nm")
print(f"Predicted Fatigue Strength: {predicted_fatigue:.2f} MPa")


Predicted Hardness: 79.19 HV
Predicted Wear Resistance: 0.169 mm³/Nm
Predicted Fatigue Strength: 269.90 MPa


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Sample dataset (Replace with integrated data from studies and experiments)
data = {
    'Al6061_fraction': [1.0, 0.95, 0.95, 0.95, 0.90],
    'B4C_fraction': [0.0, 0.05, 0.00, 0.04, 0.04],
    'Biochar_fraction': [0.0, 0.0, 0.05, 0.01, 0.06],  # Update with actual Biochar data
    'Density': [2700, 2685, 2650, 2620, 2590],  # Example values
    'Youngs_Modulus': [70, 75, 80, 85, 90],  # Example values
    'Thermal_Conductivity': [167, 160, 155, 150, 145],  # Example values
    'Tensile_Strength': [310, 320, 330, 340, 350],  # Example values
    'Compression_Strength': [420, 430, 440, 450, 460],  # Example values
    'Impact_Strength': [35, 36, 37, 38, 39],  # Example values
    'Flexural_Strength': [500, 510, 520, 530, 540]  # Example values
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Define features (X) and target variables (y)
X = df[['Al6061_fraction', 'B4C_fraction', 'Biochar_fraction', 'Density', 'Youngs_Modulus', 'Thermal_Conductivity']]
y_tensile = df['Tensile_Strength']
y_compression = df['Compression_Strength']
y_impact = df['Impact_Strength']
y_flexural = df['Flexural_Strength']

# Split data into training and testing sets
X_train, X_test, y_tensile_train, y_tensile_test = train_test_split(X, y_tensile, test_size=0.2, random_state=42)
_, _, y_compression_train, y_compression_test = train_test_split(X, y_compression, test_size=0.2, random_state=42)
_, _, y_impact_train, y_impact_test = train_test_split(X, y_impact, test_size=0.2, random_state=42)
_, _, y_flexural_train, y_flexural_test = train_test_split(X, y_flexural, test_size=0.2, random_state=42)

# Train Random Forest models
model_tensile = RandomForestRegressor(n_estimators=100, random_state=42)
model_compression = RandomForestRegressor(n_estimators=100, random_state=42)
model_impact = RandomForestRegressor(n_estimators=100, random_state=42)
model_flexural = RandomForestRegressor(n_estimators=100, random_state=42)

model_tensile.fit(X_train, y_tensile_train)
model_compression.fit(X_train, y_compression_train)
model_impact.fit(X_train, y_impact_train)
model_flexural.fit(X_train, y_flexural_train)

# Function to predict mechanical properties
def predict_properties(al6061, b4c, biochar, density, modulus, conductivity):
    input_data = np.array([[al6061, b4c, biochar, density, modulus, conductivity]])
    tensile = model_tensile.predict(input_data)[0]
    compression = model_compression.predict(input_data)[0]
    impact = model_impact.predict(input_data)[0]
    flexural = model_flexural.predict(input_data)[0]
    return tensile, compression, impact, flexural

# Example Prediction
new_sample = [0.88, 0.10, 0.02, 2650, 80, 150]  # Modify as needed
predicted_tensile, predicted_compression, predicted_impact, predicted_flexural = predict_properties(*new_sample)

print(f"Predicted Tensile Strength: {predicted_tensile:.2f} MPa")
print(f"Predicted Compression Strength: {predicted_compression:.2f} MPa")
print(f"Predicted Impact Strength: {predicted_impact:.2f} J")
print(f"Predicted Flexural Strength: {predicted_flexural:.2f} MPa")


Predicted Tensile Strength: 338.10 MPa
Predicted Compression Strength: 448.10 MPa
Predicted Impact Strength: 37.81 J
Predicted Flexural Strength: 528.10 MPa


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
